In [61]:
import h5py
import torch
import glob
import numpy as np




def open_h5(path,rgb = False):
    with h5py.File(path, 'r') as h5:
            
        labl = torch.from_numpy(h5['train_id'][:,:].astype('long'))
        if rgb:
            img = h5['raw'][0:3,:,:]
        else:
            img =h5['raw'][:,:,:]
            #img = np.moveaxis(h5['raw'][:,:,:],0,-1)
        return(img,labl)
    
x,y =  open_h5(glob.glob('processed-data/dsen_2_256/timeperiod1/STOCKHOLM/33VXF/PATCH_54424_T33VXF_20180628T101031_STOCKHOLM_256.nc')[0])

In [1]:
import os 
import glob
import zipfile as zf
import glob


paths = glob.glob('..path/to/*.zip')



for path in paths:
    print(path)
    files = zf.ZipFile(path, 'r')
    files.extractall('topath')
    files.close()


In [ ]:
### To create PERC (not used for now)
# zeros out all values not in 2 to 98 percentile

import numpy as np
import glob
import os
from osgeo import gdal, gdal_array
import pandas as pd
paths = glob.glob('../dsen2_results/timeperiod1/*.tif')
bandn =['B4',
 'B3',
 'B2',
 'B8',
 'SRB5',
 'SRB6',
 'SRB7',
 'SRB8A',
 'SRB11',
 'SRB12']    
dat = []
for path in paths:
    dataset = gdal.Open(path)
    arr = dataset.ReadAsArray()
    out = np.zeros_like(arr).astype(np.float64)
    outpath = os.path.splitext(path)[0]+ '_PERC'+ os.path.splitext(path)[1]
    name  = os.path.basename(path.split('_')[6])
    
    print(outpath)
    # calc stats 
    perc98 = np.percentile(arr,98,axis=(1,2))
    perc02 = np.percentile(arr,2,axis=(1,2))
    # print stats
    #print('maxp=\n',np.round(perc98,2))
    #print('minp=\n',np.round(perc02,2))
    
    #save stats to dict
    data = {'name':name}
    for i,band in enumerate(bandn):
        rng = (perc02[i],perc98[i])
        data.update({band:rng})
    #append dict to list     
    dat.append(data)
    
    #filter out values
    sel = np.logical_and(arr>=perc02[:,None,None],arr<=perc98[:,None,None])
    out = np.where(sel,arr,out)
    
    # save new raster
    gdal_array.SaveArray(out, outpath, "gtiff", prototype=dataset)
    
#save list of dicts to dataframe    
df = pd.DataFrame.from_dict(dat)